In [20]:
# https://www.datacareer.de/blog/accessing-ecb-exchange-rate-data-in-python/
# https://sdw-wsrest.ecb.europa.eu/help/

# airflow https://blog.freetrade.io/how-we-simplified-our-data-pipeline-54f377fad3c

# existing pandas based airflow C:\git\index2\af\dags\py\load_fx_prices.py
# existing non-pandas version   C:\gAirbusLaptopBak\AUDBackup\CustomFolders\git\index2Stash\gcf\__Need Folders for These\fxrates-get.py

In [21]:
import requests     # 2.18.4
from datetime import datetime

In [22]:
b = 'C:\\Users\\PaulDesmond\\OneDrive - SKYTRA LIMITED\\Automation\\ipy\\data\\'
f = 'to_data.csv'

dte_as_str = '2020-08-10'

# b = 'gs://pd-fxrates'
# f = '/to/data.csv'

In [23]:
# open for read here so we can set row_write to previous line straight away if necessary

# with open( os.join('data','to_data.csv')) as fin:
with open(b + f) as fin:    
    csv = fin.read()

In [24]:
print( csv[-100:] )

19/07/2020,1.1428,1.1428
20/07/2020,1.1448,1.1448
21/07/2020,1.1443,1.1443
22/07/2020,1.1578,1.1578



In [25]:
#*********************
# ECB
#*********************

In [26]:
entrypoint = 'https://sdw-wsrest.ecb.europa.eu/service/' # Using protocol 'https'
resource = 'data'           # The resource for data queries is always'data'
flowRef ='EXR'              # Dataflow describing the data that needs to be returned, exchange rates in this case
key = 'D.USD.EUR.SP00.A'    # Defining the dimension values, explained below

parameters = {
    'startPeriod': dte_as_str,  
    'endPeriod':   dte_as_str   
}

In [27]:
# Construct the URL: https://sdw-wsrest.ecb.europa.eu/service/data/EXR/D.USD.EUR.SP00.A
request_url = entrypoint + resource + '/'+ flowRef + '/' + key
response = requests.get(request_url, params=parameters, headers={'Accept': 'text/csv'})

In [40]:
# print(response)
print(response.url)
print(len(response.text))

http://www.bankofengland.co.uk/boeapps/iadb/fromshowcolumns.asp?csv.x=yes&Datefrom=10%2FAug%2F2020&Dateto=10%2FAug%2F2020&SeriesCodes=XUDLERD&CSVF=TN&UsingCodes=Y&VPD=Y&VFD=N
14


In [30]:
idx_dte = 6
idx_val = 7

if len(response.text) == 0:
    #then assume weekend or holiday and we need to use PREVIOUS value from 'csv'
    dte_str = datetime.datetime.strptime(dte, '%Y-%m-%d').strftime('%d/%m/%Y')
    row_write = ''.join( csv.split('\n')[-2:] ) + '\n'
    row_write = dte_str + row_write[10:]
else:
    lin_split = response.text.split('\n')
    row_split = lin_split[1].split(',')
    
    # format date from yyyy-mm-dd to dd/mm/yyyy only because how csv originally done
    dte_str = datetime.strptime(row_split[idx_dte], '%Y-%m-%d').strftime('%d/%m/%Y')
    val_str = row_split[idx_val]
    
    # continues for next 2 cells which we split for debugging

In [39]:
#*********************
# BOE
#*********************

# dte_as_str = '2020-08-10'

dte_boe = datetime.strptime(dte_as_str, '%Y-%m-%d').strftime('%d/%b/%Y')

url_endpoint = 'http://www.bankofengland.co.uk/boeapps/iadb/fromshowcolumns.asp?csv.x=yes'
payload = {
    'Datefrom'   : dte_boe,
    'Dateto'     : dte_boe,
    'SeriesCodes': 'XUDLERD',
    'CSVF'       : 'TN',
    'UsingCodes' : 'Y',
    'VPD'        : 'Y',
    'VFD'        : 'N'
}
response_boe = requests.get(url_endpoint, params=payload)

"""
print(response_boe.url)
print(response_boe.status_code)
print(type(response_boe.content))
print(len(response_boe.content))
print(response_boe.content.decode())
"""
# END BOE rate request

# make BOE section more robust than orig in case some decimal place change surprise
#   orig -  val_str_boe = response.content[-8:-2].decode()
# response.content sample -  b'DATE,XUDLERD\r\n06 Aug 2020,0.8439\r\n'
# list_split1 sample -  ['DATE,XUDLERD', '06 Aug 2020,0.8439', '']
list_split1 = response_boe.content.decode().split('\r\n') 
val_str_boe = list_split1[1].split(',')
print(val_str_boe)
if len(val_str_boe) == 1:
    boe_str = val_str
else:
    # val_str_boe = list_split1[1].split(',')[1]
    boe_float = round(1 / float(val_str_boe[1]), 4)
    boe_str = str(boe_float)
# END BOE rate transform

print(boe_str)

['']
1.1763


In [1]:
import requests     # 2.18.4
from datetime import datetime

In [2]:


#*********************
# BOE
#*********************

dte_as_str = '2020-08-24'

dte_boe = datetime.strptime(dte_as_str, '%Y-%m-%d').strftime('%d/%b/%Y')

url_endpoint = 'http://www.bankofengland.co.uk/boeapps/iadb/fromshowcolumns.asp?csv.x=yes'
payload = {
    'Datefrom'   : dte_boe,
    'Dateto'     : dte_boe,
    'SeriesCodes': 'XUDLERD',
    'CSVF'       : 'TN',
    'UsingCodes' : 'Y',
    'VPD'        : 'Y',
    'VFD'        : 'N'
}
response_boe = requests.get(url_endpoint, params=payload)


print(response_boe.url)
print(response_boe.status_code)
print(type(response_boe.content))
print(len(response_boe.content))
print(response_boe.content.decode())


http://www.bankofengland.co.uk/boeapps/iadb/fromshowcolumns.asp?csv.x=yes&Datefrom=24%2FAug%2F2020&Dateto=24%2FAug%2F2020&SeriesCodes=XUDLERD&CSVF=TN&UsingCodes=Y&VPD=Y&VFD=N
500
<class 'bytes'>
0



In [38]:
print(list_split1)
val_str_boe = list_split1[1].split(',')
print(val_str_boe)

['DATE,XUDLERD', '']
['']


In [32]:
row_write = ','.join([dte_str, val_str, boe_str]) + '\n'

In [33]:
with open( b + f, 'w') as fout:
    fout.write(csv + row_write)

In [34]:
# print(csv[-200:])
print( (csv + row_write)[-125:] )

19/07/2020,1.1428,1.1428
20/07/2020,1.1448,1.1448
21/07/2020,1.1443,1.1443
22/07/2020,1.1578,1.1578
10/08/2020,1.1763,1.1763

